In [1]:
import tensorflow as tf
import numpy as np
tf.random.set_seed(5)

In [2]:
xy = np.loadtxt('boston_train.csv', delimiter=',',skiprows=1,dtype=np.float32)

In [3]:
xy

array([[2.30040e+00, 0.00000e+00, 1.95800e+01, ..., 4.03000e+02,
        1.47000e+01, 2.38000e+01],
       [1.33598e+01, 0.00000e+00, 1.81000e+01, ..., 6.66000e+02,
        2.02000e+01, 1.27000e+01],
       [1.27440e-01, 0.00000e+00, 6.91000e+00, ..., 2.33000e+02,
        1.79000e+01, 2.66000e+01],
       ...,
       [1.38799e+00, 0.00000e+00, 8.14000e+00, ..., 3.07000e+02,
        2.10000e+01, 1.32000e+01],
       [7.36711e+00, 0.00000e+00, 1.81000e+01, ..., 6.66000e+02,
        2.02000e+01, 1.10000e+01],
       [1.41500e-01, 0.00000e+00, 6.91000e+00, ..., 2.33000e+02,
        1.79000e+01, 2.53000e+01]], dtype=float32)

In [4]:
x_train = xy[:,:-1]
y_train = xy[:,[-1]]
print(x_train.shape, y_train.shape)

(400, 9) (400, 1)


In [5]:
# X * W = Y
# (m,n) * (n,l) = (m,l)
# (400, 5) * (5,1) = (400,1)

In [6]:
W = tf.Variable(tf.random.normal([9,1]),name='weight')
b = tf.Variable(tf.random.normal([1]),name='bias')
print(W)
print(b)

<tf.Variable 'weight:0' shape=(9, 1) dtype=float32, numpy=
array([[-0.18030666],
       [-0.95028627],
       [-0.03964049],
       [-0.7425406 ],
       [ 1.3231523 ],
       [-0.61854804],
       [ 0.8540664 ],
       [-0.08899953],
       [ 2.4488697 ]], dtype=float32)>
<tf.Variable 'bias:0' shape=(1,) dtype=float32, numpy=array([0.22652863], dtype=float32)>


In [7]:
def hypo(X):
    return tf.matmul(X,W) +b #내적 곱셈

In [8]:
# 비용 함수 : (Hx - y)^2의 평균
# tf.square()
def cost_func():
    cost = tf.reduce_mean(tf.square(hypo(x_train) - y_train))
    return cost

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [10]:
# 학습 시작
print('***** Start Learning!!')
for step in range(10000):
    with tf.GradientTape() as tape:
        cost = cost_func()
    gradients = tape.gradient(cost, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))
    
    if step % 100 == 0:
        print('%04d' % step, '손실 값: [', cost.numpy(), ']',
              ' W:', W.numpy(), ' b:', b.numpy())
print('***** Learning Finished!!')

***** Start Learning!!
0000 손실 값: [ 4021.8206 ]  W: [[-0.17030665]
 [-0.9402863 ]
 [-0.0296405 ]
 [-0.7325406 ]
 [ 1.3331523 ]
 [-0.60854805]
 [ 0.86406636]
 [-0.07899953]
 [ 2.4588697 ]]  b: [0.23652864]
0100 손실 값: [ 230.25034 ]  W: [[-0.15476926]
 [-0.31377742]
 [ 0.00398452]
 [-0.60582733]
 [ 1.5075017 ]
 [-0.4209615 ]
 [ 0.9844709 ]
 [-0.00817521]
 [ 2.5139847 ]]  b: [0.36212817]
0200 손실 값: [ 157.94258 ]  W: [[-0.04703178]
 [-0.13201119]
 [ 0.0094742 ]
 [-0.60902196]
 [ 1.4821295 ]
 [-0.27699775]
 [ 0.6235119 ]
 [-0.02565848]
 [ 2.3416998 ]]  b: [0.27564687]
0300 손실 값: [ 119.436424 ]  W: [[-0.01426036]
 [-0.03600126]
 [ 0.01243847]
 [-0.63644195]
 [ 1.4332793 ]
 [-0.17521016]
 [ 0.19860902]
 [-0.03165422]
 [ 2.127358  ]]  b: [0.15373822]
0400 손실 값: [ 96.92389 ]  W: [[-0.05111112]
 [ 0.03484004]
 [-0.00284971]
 [-0.6663554 ]
 [ 1.4074287 ]
 [-0.11243326]
 [-0.18458678]
 [-0.03038936]
 [ 1.9173867 ]]  b: [0.04282528]
0500 손실 값: [ 83.634514 ]  W: [[-0.10528348]
 [ 0.08501689]
 [-0.033

In [11]:
print("weight = ", W.numpy)
print("bias = ", b.numpy)

weight =  <bound method BaseResourceVariable.numpy of <tf.Variable 'weight:0' shape=(9, 1) dtype=float32, numpy=
array([[-1.6099776e-01],
       [ 4.3477707e-02],
       [-1.2476283e-01],
       [-1.1958750e+01],
       [ 6.7729859e+00],
       [-4.7545187e-02],
       [-1.5368315e+00],
       [-3.0553585e-03],
       [-8.0209345e-01]], dtype=float32)>>
bias =  <bound method BaseResourceVariable.numpy of <tf.Variable 'bias:0' shape=(1,) dtype=float32, numpy=array([12.779637], dtype=float32)>>


In [12]:
test = np.loadtxt('boston_test.csv', delimiter=',',skiprows=1,
                  dtype=np.float32)
x_test = test[:,:-1]
y_test = test[:,[-1]]

In [13]:
# 정확도 측정 : RMSE
def get_rmse(y_test,preds):
    squared_error = 0
    for k,_ in enumerate(y_test):
        squared_error += (preds[k] - y_test[k])**2
    mse = squared_error/len(y_test)  
    rmse = np.sqrt(mse)
    return rmse[0]

preds = hypo(x_test).numpy()
print ('preds:',preds)
print('Rmse:',get_rmse(y_test,preds))

preds: [[21.569956]
 [33.066868]
 [22.79662 ]
 [31.093979]
 [33.690777]
 [24.256327]
 [23.397089]
 [13.452868]
 [27.854994]
 [20.9697  ]
 [26.341652]
 [18.385529]
 [19.987185]
 [41.699493]
 [14.092067]
 [15.943217]
 [27.284216]
 [17.584751]
 [28.723705]
 [17.591272]
 [16.647026]
 [22.572441]
 [24.863173]
 [17.958035]
 [17.592808]
 [21.732388]
 [30.390553]
 [36.61959 ]
 [23.888157]
 [21.041943]
 [22.497795]
 [21.582994]
 [31.756966]
 [32.996155]
 [33.313778]
 [21.53597 ]
 [20.08001 ]
 [38.636295]
 [21.302898]
 [25.67917 ]
 [22.768616]
 [21.720451]
 [15.260244]
 [20.92976 ]
 [21.620716]
 [15.833823]
 [16.064976]
 [21.2351  ]
 [17.361   ]
 [20.10188 ]
 [ 7.551177]
 [14.541158]
 [21.713312]
 [22.318262]
 [25.686005]
 [23.493202]
 [16.954086]
 [17.093952]
 [28.007133]
 [19.744074]
 [19.355358]
 [16.940231]
 [23.301945]
 [30.041729]
 [16.315235]
 [14.453145]
 [31.294079]
 [23.450768]
 [29.185532]
 [28.564285]
 [26.86196 ]
 [22.845911]
 [20.596336]
 [20.520971]
 [27.01487 ]
 [19.484577]
 [34.